<a href="https://colab.research.google.com/github/soumyadip1995/language-models/blob/main/Notebook/Multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
words = open('/content/text.txt', 'r').read().split()
words[:20]

['So',
 'Ray',
 'J',
 'went',
 'straight',
 'to',
 'the',
 'radio',
 'station',
 'The',
 'very',
 'next',
 'day,',
 '"Hey',
 'Fab,',
 "I'ma",
 'kill',
 'you!"',
 'Lyrics',
 "comin'"]

In [23]:
import torch
from torch import nn

chars = sorted(list(set(words)))
string2integer = {ch: i for i, ch in enumerate(chars)}
print(string2integer)

integer2string = {i:ch for ch,i in string2integer.items()}
encode = lambda s: [string2integer[c] for c in s]
print(encode)

decode = lambda l: ''.join([integer2string[i] for i in l])
print(decode)

data = torch.tensor(encode(words), dtype = torch.long)
print(data)
data.size()

{'"': 0, '"Hey': 1, '"It\'s': 2, '"Lose': 3, '"Oh,': 4, "'Cause": 5, "'cause": 6, "'em": 7, '(J.J.': 8, 'Doc': 9, 'Fab,': 10, 'Fad)': 11, 'For': 12, 'How': 13, 'I': 14, "I'm": 15, "I'ma": 16, 'Innovative': 17, 'J': 18, 'Lyrics': 19, 'Never': 20, 'Ray': 21, 'So': 22, 'The': 23, 'Throw': 24, 'Uh,': 25, 'Well,': 26, 'What': 27, 'With': 28, 'You': 29, 'Yourself"': 30, 'a': 31, 'and': 32, 'anything': 33, 'are': 34, "assumin'": 35, 'at': 36, 'audience': 37, 'be': 38, 'can': 39, 'celebrating': 40, "comin'": 41, 'confuse': 42, 'day': 43, 'day,': 44, 'demonstrating': 45, 'devastating,': 46, 'do': 47, 'dooma-lumma,': 48, 'elevating': 49, 'elevator': 50, 'ever': 51, 'fading,': 52, 'feeling': 53, 'fell': 54, 'forever': 55, 'found': 56, 'fuse': 57, 'get': 58, 'give': 59, 'glue': 60, 'gotta': 61, 'haters': 62, "he's": 63, 'hella': 64, 'hip-hop,': 65, 'human': 66, 'is': 67, 'it': 68, "it'll": 69, "it's": 70, 'jealous,': 71, 'kill': 72, 'know': 73, 'levitating': 74, 'like': 75, 'lose': 76, 'made': 77,

torch.Size([172])

In [24]:

block_size = 64
batch_size = 512
ix = torch.randint(len(data) - block_size, (batch_size,))
ix.size()


torch.Size([512])

In [37]:
vocab_size = len(chars)

d_k = 512

token_emb = nn.Embedding(vocab_size, d_k)
token_emb

Embedding(125, 512)

In [38]:
x = torch.stack([data[i:i + block_size] for i in ix])
x.shape
# d = x.float()
# d


torch.Size([512, 64])

In [39]:
input_embeds = token_emb(x)
input_embeds.size()

torch.Size([512, 64, 512])

In [ ]:

# p = x.transpose(-2, 1)
# p.size()

torch.Size([64, 512])

In [40]:
import math
import numpy as np 
import torch.nn.functional  as F
import torch
from math import sqrt


def scaled_dot_product(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(-2, -1)) / sqrt(dim_k)
  weights = F.softmax(scores, dim = -1)
  return torch.bmm(weights, value)
  


In [54]:
# size = 64
# dim_k = 512
# key = torch.randn(batch_size, size, dim_k)
# value = torch.randn(batch_size, size, dim_k) 
# query = torch.randn(batch_size, size, dim_k)

key = input_embeds
query = input_embeds
value = input_embeds

sdp = scaled_dot_product(query, key, value)
sdp.size()

torch.Size([512, 64, 512])

In [ ]:
# import torch
# chars = sorted(list(set(words)))
# # print(chars)
# size = len(chars) # sequence length
# # print(size)

# torch.manual_seed(1337)



# key , value, query = input_embeds

# # key = torch.randn(batch_size, size, dim_k)
# # value = torch.randn(batch_size, size, dim_k) 
# # query = torch.randn(batch_size, size, dim_k)



# sdp = scaled_dot_product_attention(query, key, value)
# sdp.size()

In [55]:
## A single attention head

class AttentionHead(nn.Module):
  def __init__(self, embedded_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embedded_dim, head_dim)
    self.k = nn.Linear(embedded_dim,  head_dim)
    self.v = nn.Linear(embedded_dim,  head_dim)

  def forward(self, x):
    attention_outputs = scaled_dot_product(self.q(x), self.k(x), self.v(x))
    

    return attention_outputs
    

### Multi headed attention

Having many heads allows the model to focus on different parts of the sentences. The softmax on one head tends to focus on one aspect of similarity. For example subject verb interaction.

In [56]:
# embedding_dim = embedding dimensions
# num_heads  = number of heads 


class MultiHeadAttention(nn.Module):
  def __init__(self, embedded_dim, num_heads):
    super().__init__()
    self.embedded_dim = embedded_dim
    self.num_heads = num_heads
    head_dim = embedded_dim // num_heads 

    self.heads = nn.ModuleList([AttentionHead(embedded_dim, head_dim) for _ in range(num_heads)])
    self.output_linear = nn.Linear(embedded_dim, embedded_dim)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    
    out = self.output_linear(out)

    return out

In [57]:
multihead_attention = MultiHeadAttention(512, 8)
# multihead_attention

attention_outputs =  multihead_attention(input_embeds)
attention_outputs.size()

torch.Size([512, 64, 512])

###### ⬆ A discrepancy in shape is occuring in the above snippet. Will try to fix it.

Note:-  Discrepancy has been solved.

An alternative implementation of Multi headed attention has also been provided.





In [58]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        d_k = K.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k).float())
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        weights = nn.Softmax(dim=-1)(scores)
        output = torch.matmul(weights, V)
        return output
        
    def forward(self, Q, K, V, mask=None):
        
        
        # Apply linear transformation to Q, K, and V
        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)
        
        # Split into multiple heads
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k)
        K = K.view(batch_size, -1, self.num_heads, self.d_k)
        V = V.view(batch_size, -1, self.num_heads, self.d_k)
        
        # Transpose to prepare for matrix multiplication
        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)
        
        # Compute attention scores and weights
        output = self.scaled_dot_product_attention(Q, K, V, mask=mask)
        
        # Concatenate the outputs of the multiple heads
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        output = self.W_o(output)
        
        return output



In [59]:
# d_model = 512
# num_heads = 8

import torch
chars = sorted(list(set(words)))
# print(chars)
seq_len = len(chars) # sequence length



Q = input_embeds
K = input_embeds
V = input_embeds

# Q = torch.randn(batch_size, seq_len, d_model)
# K = torch.randn(batch_size, seq_len, d_model)
# V = torch.randn(batch_size, seq_len, d_model)

# Create an instance of the multi-head attention model
multihead_attn = MultiHeadAttention(d_model, num_heads)

# Pass the input tensors through the model
output = multihead_attn(Q, K, V)

# The output tensor has shape (batch_size, seq_len, d_model)
print(output.shape)


torch.Size([512, 64, 512])


This one works, for now.

In [ ]:
# embedding_dim = embedding dimensions
# num_heads  = number of heads 


class MultiHeadAttention(nn.Module):
  def __init__(self, embedded_dim, num_heads):
    super().__init__()
    self.embedded_dim = embedded_dim
    self.num_heads = num_heads
    head_dim = embedded_dim // num_heads 

    self.heads = nn.ModuleList([AttentionHead(embedded_dim, head_dim) for _ in range(num_heads)])
    self.output_linear = nn.Linear(embedded_dim, embedded_dim)

  def forward(self, p):
    out = torch.cat([h(p) for h in self.heads], dim = -1)
    
    out = self.output_linear(out)

    return out


In [ ]:
multihead_attention = MultiHeadAttention(embedded_dim = 512, num_heads = 8)
multihead_attention

MultiHeadAttention(
  (heads): ModuleList(
    (0): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (1): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (2): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (3): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (4): AttentionHead(
      (q): Linear(in_features=512, out_featur

In [ ]:
#### Single headed attention

import torch
import torch.nn as nn

class SingleAttention(nn.Module):
    def __init__(self, embedded_dim):
        super(SingleAttention, self).__init__()
        self.embedded_dim = embedded_dim
        
        self.W_q = nn.Linear(embedded_dim, embedded_dim)
        self.W_k = nn.Linear(embedded_dim, embedded_dim)
        self.W_v = nn.Linear(embedded_dim, embedded_dim)
        self.W_o = nn.Linear(embedded_dim, embedded_dim)
        
    def forward(self, Q, K, V, mask=None):
        
        
        # Apply linear transformation to Q, K, and V
        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)
        
        # Compute attention scores and weights
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.embedded_dim).float())
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        weights = nn.Softmax(dim=-1)(scores)
        
        # Apply attention weights to values
        output = torch.matmul(weights, V)
        output = self.W_o(output)
        
        return output


In [ ]:



import torch
chars = sorted(list(set(words)))
# print(chars)
seq_len = len(chars) # sequence length
embedded_dim = 512


# Create some dummy input tensors
Q = torch.randn(batch_size, seq_len, embedded_dim)

K = torch.randn(batch_size, seq_len, embedded_dim)
V = torch.randn(batch_size, seq_len, embedded_dim)

# Create an instance of the single attention model
single_attn = SingleAttention(embedded_dim)

# Pass the input tensors through the model
output = single_attn(Q, K, V)
output.shape


torch.Size([512, 125, 512])